In [ ]:
#This code is from:  https://blog.mlq.ai/deep-dream-with-tensorflow-2-0/

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
names = ['mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]
deepdream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

In [ ]:
Sample_Image = tf.keras.preprocessing.image.load_img(r'starry.jpg', target_size=(225, 375))
Sample_Image = np.array(Sample_Image)#/255.0
Sample_Image = tf.keras.preprocessing.image.img_to_array(Sample_Image)
Sample_Image = tf.Variable(tf.keras.applications.inception_v3.preprocess_input(Sample_Image))

In [ ]:
def calc_loss(image, model):
  # convert to batch format
  img_batch = tf.expand_dims(image, axis=0)
  # run the model
  layer_activations = model(img_batch)

  # create an empty array to accumulate the losses
  losses = []
  for act in layer_activations:
    # calculate the mean of each activation
    loss = tf.math.reduce_mean(act)
    losses.append(loss)

  # calcualte sum
  return tf.reduce_sum(losses)

In [ ]:
@tf.function
def deepdream(model, image, step_size):
    with tf.GradientTape() as tape:
      tape.watch(image)
      loss = calc_loss(image, model) 

    gradients = tape.gradient(loss, image)

    gradients /= tf.math.reduce_std(gradients)  

    image = image + gradients * step_size
    image = tf.clip_by_value(image, -1, 1)

    return loss, image

In [ ]:
def run_deep_dream_simple(model, image, steps=100, step_size=0.01):
  #image = tf.keras.applications.inception_v3.preprocess_input(image)

  for step in range(steps):
    loss, image = deepdream(model, image, step_size)
    
    if step % 1000 == 0:
      plt.figure(figsize=(12,12))
      plt.imshow(deprocess(image))
      plt.show()
      print ("Step {}, loss {}".format(step, loss))

  plt.figure(figsize=(12,12))
  plt.imshow(deprocess(image))
  plt.show()

  return deprocess(image)

In [ ]:
def deprocess(image):
  image = 255*(image + 1.0)/2.0
  return tf.cast(image, tf.uint8)

In [ ]:
dream_img = run_deep_dream_simple(model=deepdream_model, image=Sample_Image, 
                                  steps=2000, step_size=0.001)